In [0]:
import numpy as np

In [0]:
from scipy import spatial

In [0]:
pip install sister

     |████████████████████████████████| 61kB 4.4MB/s 
     |████████████████████████████████| 21.5MB 111kB/s 
  Created wheel for sister: filename=sister-0.1.5-cp36-none-any.whl size=4232 sha256=0c18fc85a5e6cc623068d9876702f8f1f5ca7502f74100a0bdfedeafcbb0308e
  Stored in directory: /root/.cache/pip/wheels/7d/ed/48/76cdf61511496a1cd2a2c56dc9b4e9667401dfb6084f153cc3
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2383730 sha256=149ef6ca7762d530c31aae25ccf9db890eea11d6863bc7d95d5a090095d31c13
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built sister fasttext


In [0]:
import sister
embedder = sister.MeanEmbedding(lang="en")

sentence = "I am a dog."
vector = embedder(sentence) 

Loading model...


In [0]:
sen = "im a puppy"
vec = embedder(sen)

In [0]:
x = spatial.distance.cosine(vector,vec)
1-x


0.6176577210426331

In [0]:
if vector.all() == vec.all():
  print("yes")

yes


In [0]:
from numpy import dot
from numpy.linalg import norm

cos_sim = dot(vector,vec)/(norm(vector)*norm(vec))

In [0]:
cos_sim

0.6176577

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import json
import pandas as pd


In [0]:
train = pd.read_json('/content/drive/My Drive/SQuAD-master/data/train-v1.1.json')
train.shape

(442, 2)

In [0]:
contexts = []
questions = []
answers_text = []
answers_start = []
for i in range(train.shape[0]):
    topic = train.iloc[i,0]['paragraphs']
    for sub_para in topic:
        for q_a in sub_para['qas']:
            questions.append(q_a['question'])
            answers_start.append(q_a['answers'][0]['answer_start'])
            answers_text.append(q_a['answers'][0]['text'])
            contexts.append(sub_para['context'])   
df = pd.DataFrame({"context":contexts, "question": questions, "answer_start": answers_start, "text": answers_text})

In [0]:
questionEmbed = {}
for question in questions:
  questionEmbed[question] = embedder(question)

In [0]:
qembed = list(questionEmbed.values())
df.head()

,context,question,answer_start,text
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary


In [0]:
df.iloc[0]['text']

'Saint Bernadette Soubirous'

In [0]:
from scipy import spatial

In [0]:
def enterQuestion():
  question = input()
  max = 0
  answer =''
  q =""
  qe = embedder(question)
  for i in range(len(qembed)):
    x = 1 - spatial.distance.cosine(qe,qembed[i])
    if x>max:
      max = x
      q=questions[i]
      answer = df.iloc[i]['text']
  return max,answer,q



In [0]:
max,answer,q = enterQuestion() 
#enterQuestion is for questions based on wikipedia articles

whens my birthday


In [0]:
answer

'five'

In [0]:
import re

In [0]:
def clean_text(text):

    #text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)

   
    return text
    


In [0]:
def divide(para):
  sentences = re.split(r' *[\.\?!][\'"\)\]]* *', para)
  x_train = list()
  for stuff in sentences:
      x_train.append(stuff)
  return x_train

In [0]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [0]:
import pandas as pd

In [0]:
def getQA(sentences):
  cont =[]
  answ = []
  labels = []
  embed = []
  check =[]
  prev =""
  for sentence in sentences:
   # print(sentence)
    doc = nlp(sentence)
    for ent in doc.ents:
      #print(ent)
      if len(doc.ents)>1:
        check.append(True)
      else:
        check.append(False)
      cont.append(sentence)
      answ.append(str(ent))
      labels.append(str(ent.label_))
      embed.append(embedder(sentence))
  df = pd.DataFrame({'context':cont,'answers':answ,'labels':labels,'sentenceEmbed':embed,'check':check})
  return df


In [0]:
def enterPara():
  para = input()
  sent = divide(para)
  newSent =[clean_text(sent[i]) for i in range(len(sent))]
  return newSent

In [139]:
newSent = enterPara()

A small village in the Jaunpur district of Uttar Pradesh, Madhopatti has created history by producing the most number of IAS officers. With about 75 households, it has given 47 officers to Indian administration. Not just that, many from the same village have made their careers by joining prestigious organisations like ISRO, Bhabha Atomic Research Centre and World Bank. Ask the villagers about their study schedules, maybe? :p


In [0]:
df= getQA(newSent)

In [141]:
df

,context,answers,labels,sentenceEmbed,check
0,A small village in the Jaunpur district of Utt...,Jaunpur,LOC,"[0.15495865, -0.093286544, 0.028984595, -0.173...",True
1,A small village in the Jaunpur district of Utt...,Uttar Pradesh Madhopatti,ORG,"[0.15495865, -0.093286544, 0.028984595, -0.173...",True
2,With about 75 households it has given 47 offic...,about 75,CARDINAL,"[0.111352, -0.037353825, 0.09222373, -0.223540...",True
3,With about 75 households it has given 47 offic...,47,CARDINAL,"[0.111352, -0.037353825, 0.09222373, -0.223540...",True
4,With about 75 households it has given 47 offic...,Indian,NORP,"[0.111352, -0.037353825, 0.09222373, -0.223540...",True
5,Not just that many from the same village have ...,ISRO Bhabha Atomic Research Centre,ORG,"[0.17213036, -0.04548471, 0.07099308, -0.15256...",True
6,Not just that many from the same village have ...,World Bank,ORG,"[0.17213036, -0.04548471, 0.07099308, -0.15256...",True


In [0]:
whdict = {'what':['PERSON','THING','FAC','ORG'],'who':['PERSON','ORG','NORP'],'where':['PLACE','LOC','GPE'],'when':['DATE','TEMPORAL'],'how':['NUMERIC','CARDINAL']}


In [0]:
def enterQuestion1():
  question = input()
  max = 0
  answer =''
  q =""
  index =0 
  qe = embedder(question)
  whWord = question.split()[0].lower()
 
  for i in range(len(df.context)):
    x = 1-spatial.distance.cosine(qe,df.sentenceEmbed[i])
    if x>max :
      index = i
      max = x
      answer = df.answers[i]
  if df.check[index] == True:
    whLabel = whdict[whWord]
    i=index
    while(df.check[i] != False):
      if df.labels[i] in whLabel:
        answer = df.answers[i]
        break
      i+=1
  return answer
    

In [142]:
enterQuestion1()


where is this small village called Madhopatti located


'Jaunpur'

In [0]:
enterQuestion1()